In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_csv('default_report_2.csv', index_col=False)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
rbs_c = pd.DataFrame.from_csv('raw_test.csv', index_col=False)
rbs_c['Date'] = pd.to_datetime(rbs_c['Date'], dayfirst=True)
fixed_names = (name.strip() for name in rbs_c.columns)
rbs_c.rename(columns=dict(zip(rbs_c.columns, fixed_names)), inplace=True)

In [2]:
def search_by_val_and_date(bank, value_range, since_date):
    match_value = (bank['Value'] < max(value_range)) & (bank['Value'] > min(value_range))
    match_date = (bank['Date'] > pd.to_datetime(since_date, dayfirst=True))
    return bank[match_value & match_date]

In [3]:
def unique_strat(budget_row_index, budget_date, budget_amount, bank):
    """
    Checks an entry in the budget logs and sees if there is a 1:1 match with an entry in bank
    That entry in bank has to be 'unmatched'
    """
    date_match = abs(bank['Date'] - budget_date) < pd.Timedelta('4 Days')
    amount_match = bank['Value'] == budget_amount
    uncategorised = bank['Category'].notnull()  # likely to break if not using existing SOT
    matches = bank[date_match & amount_match]
    if len(matches) == 1:
        return (budget_row_index, matches.index[0])
    return False

def reconcile(budget, bank, budget_account='Rbs'):
    """
    Matches entries in budget with entries in RBS
    """
    matched_list = []
    unmatched_list = []
    for n, (date, cat, subcat, amount, acc, payee, note) in budget.iterrows():
        if acc != budget_account:
            pass
        unique_row = unique_strat(n, date, amount, bank)
        if unique_row:
            matched_list += [unique_row]
        else:
            unmatched_list += [n]
    return matched_list, unmatched_list

matched, unmatched = reconcile(df, rbs_c)

In [4]:
budget_matches, bank_matches = zip(*matched)
budget_matches

(1, 2, 3, 7, 8, 10, 11, 13, 16)

In [22]:
rbs_period = rbs_c[rbs_c['Date'] > pd.to_datetime('2015/12/09')]
matched_budget_rows = df.iloc[list(budget_matches)]
unmatched_budget_rows = df.iloc[unmatched]
matched_budget_rows.index = bank_matches
matched_bank_rows = pd.concat([rbs_period, matched_budget_rows], axis=1)

In [23]:
matched_bank_rows[matched_bank_rows['Category'].isnull()]

,Date,Type,Description,Value,Balance,Account Name,Account Number,Date,Category,SubCategory,Expense Amount,Account,Payee,Notes
248,2015-12-10,POS,"'9285 09DEC15 C , SUBWAY , CITY ROAD LON GB",-6.98,2163.01,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
249,2015-12-14,POS,"'9285 11DEC15 , GIVEWELL , 6462332035 US , USD...",-4.31,2158.70,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
250,2015-12-14,POS,"'9285 11DEC15 , BB *RAINFOREST , TRUST , 80045...",-4.31,2154.39,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
251,2015-12-14,POS,"'9285 13DEC15 , 447581427540 , FEE GB",-20.17,2134.22,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
252,2015-12-14,POS,"'9285 12DEC15 C , TFL.GOV.UK/CP , TFL TRAVEL C...",-1.50,2132.72,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
253,2015-12-14,POS,"'9285 12DEC15 C , PAPERCHASE , STRATFORD GB",-3.00,2129.72,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
255,2015-12-14,POS,"'9285 12DEC15 C , M AND S SIMPLY FOOD - , SSP ...",-8.25,2112.89,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
260,2015-12-14,POS,"'9285 12DEC15 C , CO-OP GROUP PETROL, BOW ROAD GB",-7.90,2066.13,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
261,2015-12-14,POS,"'9285 11DEC15 C , SIMPLY FRESH , LONDON GB",-12.47,2053.66,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN
264,2015-12-15,POS,"'9285 14DEC15 C , SUBWAY , CITY ROAD LON GB",-5.50,4996.15,'Current,'162632-10651269,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
matched_bank_rows[matched_bank_rows['Category'].notnull()]

,Date,Type,Description,Value,Balance,Account Name,Account Number,Date,Category,SubCategory,Expense Amount,Account,Payee,Notes
254,2015-12-14,POS,"'9285 13DEC15 C , CO-OP GROUP PETROL, BOW ROAD GB",-8.58,2121.14,'Current,'162632-10651269,2015-12-13,Food/Groceries,Groceries,-8.58,Rbs,NaN,NaN
256,2015-12-14,POS,"'9285 12DEC15 , NEWSPOINT , (STRATFORD) , LOND...",-5.17,2107.72,'Current,'162632-10651269,2015-12-12,Food/Groceries,Snack / Sweets,-5.17,Rbs,NaN,NaN
257,2015-12-14,POS,"'9285 12DEC15 C , TORTILLA WESTFIELD, STR , LO...",-16.70,2091.02,'Current,'162632-10651269,2015-12-12,Food/Groceries,Restaurant/Fast food,-16.70,Rbs,NaN,NaN
258,2015-12-14,POS,"'9285 12DEC15 C , SACAT MARKS AND , SPENCER , ...",-12.00,2079.02,'Current,'162632-10651269,2015-12-12,Entertainment,Night Out,-12.00,Rbs,NaN,NaN
259,2015-12-14,POS,"'9285 12DEC15 , LEGO STORE , STRATFORD CTY GB",-4.99,2074.03,'Current,'162632-10651269,2015-12-12,Misc/One-time,Gifts/Charity,-4.99,Rbs,NaN,NaN
262,2015-12-15,BAC,'ELSEVIER LTD,2982.97,5036.63,'Current,'162632-10651269,2015-12-16,Salary,NaN,2982.97,Rbs,NaN,NaN
263,2015-12-15,POS,"'9285 14DEC15 , CURRYS DIGITAL , LONDON GB",-34.98,5001.65,'Current,'162632-10651269,2015-12-14,Departmental,Electronics,-34.98,Rbs,NaN,NaN
266,2015-12-15,POS,"'9285 14DEC15 , CYCLE SURGERY , LONDON EC1Y GB",-69.98,4853.48,'Current,'162632-10651269,2015-12-14,Car/Auto,Bike,-69.98,Rbs,NaN,NaN
268,2015-12-18,IBP,"'9285 16DEC15 , TESCO STORES-2092",-7.50,4842.98,'Current,'162632-10651269,2015-12-16,Food/Groceries,Groceries,-7.50,Rbs,NaN,NaN


In [32]:
unmatched_budget_rows[unmatched_budget_row]

,Date,Category,SubCategory,Expense Amount,Account,Payee,Notes
0,2015-12-10,Food/Groceries,Restaurant/Fast food,-10.00,NaN,NaN,NaN
4,2015-12-14,Food/Groceries,Restaurant/Fast food,-5.29,Rbs,NaN,NaN
5,2015-12-15,Food/Groceries,Groceries,-8.99,Cash,NaN,Coconut Aminos
6,2015-12-15,Food/Groceries,Restaurant/Fast food,-6.15,Cash,NaN,Burrito
9,2015-12-10,Entertainment,Movies,-22.00,Rbs,NaN,NaN
12,2015-12-12,Misc/One-time,Gifts/Charity,-2.00,Cash,NaN,NaN
14,2015-12-12,Misc/One-time,Misc,-3.00,Rbs,NaN,Ribbon
15,2015-12-10,Pt,NaN,40.00,Cash,NaN,NaN


In [ ]:
matched_step_2 = [()]